In [ ]:
# Install required packages
!pip install shap lime transformers torch torchvision

import torch
from transformers import XLMRobertaForTokenClassification, DistilBertForTokenClassification, BertForTokenClassification
from transformers import XLMRobertaTokenizer, DistilBertTokenizer, BertTokenizer
import shap
from lime.lime_text import LimeTextExplainer
import numpy as np

# Load your models and tokenizers
model_name_xlm = 'xlm-roberta-base'  # XLM-Roberta
model_name_distil = 'distilbert-base-multilingual-cased'  # DistilBERT

tokenizer_xlm = XLMRobertaTokenizer.from_pretrained(model_name_xlm)
tokenizer_distil = DistilBertTokenizer.from_pretrained(model_name_distil)

# Load models
model_xlm = XLMRobertaForTokenClassification.from_pretrained(model_name_xlm)
model_distil = DistilBertForTokenClassification.from_pretrained(model_name_distil)

# Set models to evaluation mode
model_xlm.eval()
model_distil.eval()

# Sample tokenized input data
sample_data = ["John Smith visited New York City.", "The river bank near the bank of America was flooded."]

# Function to tokenize input
def tokenize_input(text):
    return tokenizer_xlm(text, return_tensors="pt", padding=True, truncation=True)

# SHAP Implementation
def explain_with_shap(model, text):
    # Tokenize input
    tokenized_data = tokenize_input(text)

    # Define a function to make predictions
    def predict_fn(input_ids):
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits
            return torch.softmax(logits, dim=2).numpy()  # Get probabilities

    # Create explainer
    explainer = shap.Explainer(predict_fn, tokenized_data['input_ids'])

    # Compute SHAP values
    shap_values = explainer(tokenized_data['input_ids'])

    # Plot SHAP summary
    shap.summary_plot(shap_values, feature_names=tokenizer_xlm.convert_ids_to_tokens(tokenized_data['input_ids'][0].tolist()))

# LIME Implementation
def explain_with_lime(model, text):
    # Define a function to make predictions
    def predict_fn(texts):
        predictions = []
        for text in texts:
            inputs = tokenizer_distil(text, return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                outputs = model(inputs['input_ids'])
                logits = outputs.logits
                predictions.append(torch.softmax(logits, dim=2).numpy())
        return np.array(predictions)

    explainer = LimeTextExplainer(class_names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'])
    explanation = explainer.explain_instance(text, predict_fn, num_features=10)
    explanation.show_in_notebook(text=text)

# Explain the models
print("SHAP Explanation for XLM-Roberta:")
explain_with_shap(model_xlm, sample_data[0])  # Example text for SHAP
print("LIME Explanation for DistilBERT:")
explain_with_lime(model_distil, sample_data[1])  # Example text for LIME

